# LOD-Datasets

## Ziel: Datasets mit Metadaten von LOSD extrahieren, um sie auf CKAN zu importieren.

SPARQL-Queries als Basis:
- Datasets with Observations: http://yasgui.org/short/0TZ3Oo1w0
- Observations with multiple MeasureProperty's: http://yasgui.org/short/wzt1WNUEw
- Dataset metadata (meta.xml): http://yasgui.org/short/G1pSGUyN2
- Beispiel SPARQL Query (für als Ressource): http://yasgui.org/short/DM-Xwuqi3

In [18]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pprint import pprint
import pandas as pd

def querySparql(query):
    # The ld.stadt-zuerich.ch endpoints doesn't seem to work, use LINDA instead
    #sparql = SPARQLWrapper("https://ld.stadt-zuerich.ch/query")
    sparql = SPARQLWrapper("https://lindas-data.ch:8443/lindas/query")
    sparql.setHTTPAuth("BASIC")
    sparql.setCredentials("public", "public")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

Die folgende **SPARQL-Query** holt sich **alle Datasets mit mind. einer Observation** haben (im Subgraph der Stadt Zürich):

In [19]:
allDatasets = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000
"""
print(allDatasets)


PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?dataset ?label WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {

   ?dataset a qb:DataSet ; 
      rdfs:label ?label .     
    
   # ?obs a qb:Observation ;
   ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
   
   # FILTER (?dataset = <https://ld.stadt-zuerich.ch/statistics/dataset/AVA-RAUM-ZEIT-GGH-HEL-SEX>)  
}}
GROUP BY ?dataset ?label
LIMIT 1000



In [20]:
datasetResult = querySparql(allDatasets)

Dies ist die **Liste aller Datasets**:

In [21]:
datasets = [{'uri': d['dataset']['value'], 'label': d['label']['value']} for d in datasetResult]
pprint([d['label'].encode('utf-8') for d in datasets])
print("%s Datasets" % len(datasets))

['Arbeitsst\xc3\xa4tten nach Betriebsart, Raum, Zeit',
 'Besch\xc3\xa4ftigte nach Betriebsart, Raum, Geschlecht, Zeit',
 'Besch\xc3\xa4ftigte nach Betriebsart, Raum, Zeit',
 'Zuschauer/innen, Besucher/innen nach Betriebsart, Heimatland, Raum, Zeit',
 'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Geschlecht, Zeit',
 'Zuschauer/innen, Besucher/innen nach Betriebsart, Raum, Zeit',
 'Sch\xc3\xbcler/innen und Student/innen nach Betriebsart, Raum, Schulstufen und F\xc3\xa4cher, Zeit',
 'Sch\xc3\xbcler/innen und Student/innen nach Betriebsart, Raum, Zeit',
 'Arbeitsst\xc3\xa4tten nach Bewilligung, Betriebsart, Raum, Zeit',
 'Wirtschaftliche Wohnbev\xc3\xb6lkerung nach Raum, Zeit',
 'Wirtschaftliche Wohnbev\xc3\xb6lkerung nach Raum, Geschlecht, Zeit',
 'Wirtschaftliche Wohnbev\xc3\xb6lkerung nach Heimatland, Raum, Geschlecht, Zeit',
 'Wirtschaftliche Wohnbev\xc3\xb6lkerung nach Heimatland, Raum, Zeit',
 'Anteil nach Grundgesamtheit, Heimatland, Raum, Zeit',
 'Wirtschaftliche Wohnbev

In [22]:
def generateMetaQuery(dataset):
    metaQuery = u'''
    PREFIX qb: <http://purl.org/linked-data/cube#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    WHERE { GRAPH <https://linked.opendata.swiss/graph/zh/statistics> {
       ?dataset a qb:DataSet ; 
            rdfs:label ?title .     

        # group
       OPTIONAL {
           ?category a <https://ld.stadt-zuerich.ch/schema/Category> ;
           rdfs:label ?categoryLabel ;
           skos:narrower* ?dataset .
        }

         # quelle, zeit, aktualisierungsdatum
        ?obs <http://purl.org/linked-data/cube#dataSet> ?dataset .
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/QUELLE> ?quelle .
            ?quelle rdfs:label ?quelleLabel .
        }
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/property/RAUM> ?raum .
            ?raum rdfs:label ?raumLabel .
        }
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/property/ZEIT> ?zeit } .
        OPTIONAL { ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/DATENSTAND> ?updateDate } .
        
        # evtl. GLOSSAR und BTA (und weitere) für Tags verwenden
        OPTIONAL {
            ?obs <https://ld.stadt-zuerich.ch/statistics/attribute/GLOSSAR> ?glossar .
            ?glossar rdfs:label ?glossarLabel .
        }
         OPTIONAL {
             ?obs <https://ld.stadt-zuerich.ch/statistics/property/BTA> ?bta .        
             ?bta rdfs:label ?btaLabel .
        }

       FILTER (?dataset = <DATASET_URI>)
    }}
    GROUP BY ?dataset ?title ?categoryLabel ?quelleLabel ?zeit ?updateDate ?glossarLabel ?btaLabel ?raumLabel
    LIMIT 1000'''
    metaQuery = metaQuery.replace('DATASET_URI', dataset['uri'])
    return metaQuery

def mapMetadata(dataset, metaResult):
    meta = {
        'uri': dataset['uri'],
        'title': dataset['label'],
        'description': '',
        'category': list(set([t.get('categoryLabel', {}).get('value', '') for t in metaResult])),
        'legal_basis': '',
        'spatial': list(set([t.get('raumLabel', {}).get('value', '') for t in metaResult])),
        'publisher': 'Open Data Zurich',
        'source': list(set([t.get('quelleLabel', {}).get('value', '') for t in metaResult])),
        'temporal': list(set([t.get('zeit', {}).get('value', '') for t in metaResult])),
        'quality': '',
        'published_date': list(set([t.get('zeit', {}).get('value', '') for t in metaResult])),
        'update_date': list(set([t.get('updateDate', {}).get('value', '') for t in metaResult])),
        'data_type': 'Datenaggregat',
        'periodicity': '',
        'tags': list(set([t.get('glossarLabel', {}).get('value', '') for t in metaResult])),
        'version': '',
        'remarks': '',
    }
    return meta

In [25]:
metadata = []
# only loop over first 10 datasets
for dataset in datasets[:10]:
    metaQuery = generateMetaQuery(dataset)
    # print(metaQuery)
    
    metaResult = querySparql(metaQuery)
    #pprint(metaResult[0])
    
    meta = mapMetadata(dataset, metaResult)
    metadata.append(meta)

Zeige beispielhaft **10 Datasets inkl. der Metadaten**, die CKAN braucht (_meta.xml_):

In [24]:
pd.DataFrame(data=metadata)

,category,data_type,description,legal_basis,periodicity,published_date,publisher,quality,remarks,source,spatial,tags,temporal,title,update_date,uri,version
0,[],Datenaggregat,,,,"[1944-12-31, 1981-12-31, 1994-12-31, 2006-12-3...",Open Data Zurich,,,"[Sicherheitsdepartement, Stadtpolizei Zürich, ...","[City, Oerlikon, Unterstrass, Enge, Schwamendi...","[Verpflegungsbetriebe, Horte]","[1944-12-31, 1981-12-31, 1994-12-31, 2006-12-3...","Arbeitsstätten nach Betriebsart, Raum, Zeit",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
1,[],Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Geschlecht, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Beschäftigte nach Betriebsart, Raum, Geschlech...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
2,"[Bildung, Kultur & Sport]",Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Zentralbibliothek Zürich, Schul- und Sportdep...",[Stadt Zürich (ab 1934)],"[, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Beschäftigte nach Betriebsart, Raum, Zeit","[2018-08-08, 2018-08-21]",https://ld.stadt-zuerich.ch/statistics/dataset...,
3,[],Datenaggregat,,,,"[1986-06-30, 1980-06-30, 1972-06-30, 1978-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Horte, Heimatland]","[1986-06-30, 1980-06-30, 1972-06-30, 1978-06-3...","Zuschauer/innen, Besucher/innen nach Betriebsa...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
4,[],Datenaggregat,,,,"[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...",Open Data Zurich,,,"[Schul- und Sportdepartement, Schulamt]",[Stadt Zürich (ab 1934)],"[Geschlecht, Horte]","[2014-06-30, 1986-06-30, 1980-06-30, 1972-06-3...","Zuschauer/innen, Besucher/innen nach Betriebsa...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
5,"[Bildung, Kultur & Sport]",Datenaggregat,,,,"[1976, 1997, 1962, 1960, 1983-06-30, 1961, 198...",Open Data Zurich,,,"[Zoo Zürich AG, Museums- und Sammlungsverwaltu...",[Stadt Zürich (ab 1934)],"[, Oper, Horte, Theater]","[1976, 1997, 1962, 1960, 1983-06-30, 1961, 198...","Zuschauer/innen, Besucher/innen nach Betriebsa...","[2018-08-22, 2018-08-21, 2018-08-08]",https://ld.stadt-zuerich.ch/statistics/dataset...,
6,[],Datenaggregat,,,,"[2016, 2015, 2014, 2017, 2009, 2011, 2010, 201...",Open Data Zurich,,,"[Schul- und Sportdepartement, Musikschule Kons...","[Schulkreis Glattal, Schulkreis Limmattal, Sch...","[Jugend- und Erwachsenenbildung, Musikinstrument]","[2016, 2015, 2014, 2017, 2009, 2011, 2010, 201...",Schüler/innen und Student/innen nach Betriebsa...,[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
7,[],Datenaggregat,,,,"[2016, 2015, 2014, 2017, 2009, 2011, 2010, 201...",Open Data Zurich,,,"[Schul- und Sportdepartement, Musikschule Kons...","[Schulkreis Glattal, Schulkreis Limmattal, Sch...",[Jugend- und Erwachsenenbildung],"[2016, 2015, 2014, 2017, 2009, 2011, 2010, 201...",Schüler/innen und Student/innen nach Betriebsa...,[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
8,[],Datenaggregat,,,,"[1994-12-31, 2006-12-31, 2001-12-31, 1998-12-3...",Open Data Zurich,,,"[Sicherheitsdepartement, Stadtpolizei Zürich, ...","[City, Friesenberg, Rathaus, Kreis 8 (ab 1915)...",[Verpflegungsbetriebe],"[1994-12-31, 2006-12-31, 2001-12-31, 1998-12-3...","Arbeitsstätten nach Bewilligung, Betriebsart, ...",[2018-08-08],https://ld.stadt-zuerich.ch/statistics/dataset...,
9,[Bevölkerung],Datenaggregat,,,,"[1904-12-31, 1921-12-31, 1957-12-31, 1919-12-3...",Open Data Zurich,,,"[Statistik Stadt Zürich, BVS, Statistik Stadt ...","[alter Kreis 7 (1915-1933), City, Schwamending...",[],"[1904-12-31, 1921-12-31, 1957-12-31, 1919-12-3...","Wirtschaftliche Wohnbevölkerung nach Raum, Zeit",[2018-08-17],https://ld.stadt-zuerich.ch/statistics/dataset...,


## Fazit

- **Es ist möglich, die meisten Metadaten aus LOSD zu extrahieren**
- Es gibt auch Dinge die aktuell **fehlen** (siehe leere Felder oben, z.B. _legal_basis_, _description_)
- Da im LOSD viele Metadaten an den Observations hängen, gibt es potentiell **mehrere Werte für einige Felder** (z.B. _published_date_, _source_, _spatial_). Hier braucht es entweder eine Änderung in CKAN oder eine Logik für die Auswahl des "korrekten" Wertes.
- **Nicht alle Dataset haben eine Kategorie** (ist dies korrekt oder ein Fehler?)


**Noch offen:**
- Generieren von **Beispiel-SPARQL-Queries als Ressourcen** (siehe z.B. http://yasgui.org/short/DM-Xwuqi3)
- Generieren von Attribut-Listen (wahrscheinlich nicht sinnvoll, da diese Information bereits im SPAQL-Query "encoded" ist)